Malayalam English Dataset

### Imports

In [33]:
!pip install torchtext

You should consider upgrading via the '/home/ckm/python3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
import os
import math
import numpy as np
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from time import sleep


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
!nvidia-smi

Mon Nov 15 11:03:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:17:00.0 Off |                    0 |
| N/A   56C    P0    45W / 250W |    311MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load embedding

In [37]:
datapath = "/home/ckm/ck_project/data/MalayalmEnglishSentiments/"
df_embedding = pd.read_csv(datapath+"mal_eng_emb", sep=" ", quoting=3, header=None, index_col=0,skiprows=1)

In [38]:
df_embedding.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
like,-0.071869,0.275370,-0.071923,0.032835,0.199517,0.160000,-0.159244,0.026283,0.343170,0.352353,...,0.145661,0.047930,0.269013,-0.226423,-0.122173,-0.181215,0.034296,0.128978,-0.141020,-0.143799
oru,-0.077830,0.266342,-0.074621,0.033034,0.198433,0.159149,-0.159517,0.021624,0.346139,0.352183,...,0.139258,0.042647,0.274619,-0.225015,-0.124942,-0.182240,0.036177,0.136150,-0.133971,-0.136864
trailer,-0.074339,0.276567,-0.075053,0.036022,0.206773,0.164624,-0.159937,0.022718,0.350942,0.356257,...,0.146939,0.046356,0.279289,-0.230388,-0.132882,-0.184771,0.033757,0.134053,-0.137140,-0.141513
fans,-0.076445,0.272194,-0.076098,0.033571,0.199238,0.159381,-0.161973,0.020347,0.344558,0.354843,...,0.140745,0.042472,0.277846,-0.223677,-0.124965,-0.186585,0.031947,0.131690,-0.138251,-0.138841
ikka,-0.073600,0.267904,-0.076091,0.032226,0.196860,0.157712,-0.166063,0.018810,0.339874,0.351412,...,0.135749,0.043580,0.276364,-0.222802,-0.128295,-0.188228,0.034005,0.132371,-0.137775,-0.142165


### Create vocab dictionary

In [39]:
m = df_embedding.to_numpy()
m.shape

(1109, 100)

In [40]:
embedding_matrix = df_embedding.to_numpy()

vocab = []

for word in list(df_embedding.index):
  vocab.append(str(word))

vocab_size , vocab_dim = embedding_matrix.shape
vocab_size, vocab_dim

(1109, 100)

In [41]:
len(vocab)

1109

In [42]:
word2idx = {w: idx for (idx, w) in enumerate(vocab)}
idx2word = {idx: w for (idx, w) in enumerate(vocab)}

In [43]:
len(word2idx) , len(idx2word)

(1109, 1109)

In [44]:
import itertools
dict(itertools.islice(word2idx.items(), 10))

{'like': 0,
 'oru': 1,
 'trailer': 2,
 'fans': 3,
 'ikka': 4,
 'padam': 5,
 'mass': 6,
 'Trailer': 7,
 'movie': 8,
 'waiting': 9}

### Read data

In [45]:
df = pd.read_csv('/home/ckm/ck_project/data/MalayalmEnglishSentiments/Malayalam_English_Data.csv')

df= df.sample(frac = 1)

train_df = df[:4000]
val_df = df[4000:4400]
test_df = df[4400:]

train_df.head()

train_data =  train_df['text'].values
train_labels = train_df['category'].values

val_data =  val_df['text'].values
val_labels = val_df['category'].values

test_data =  test_df['text'].values
test_labels = test_df['category'].values

print(len(train_data), len(train_labels))
print(len(val_data), len(val_labels))
print(len(test_data), len(test_labels))


4000 4000
400 400
451 451


In [46]:
df.head()

,Unnamed: 0,text,category
3637,3637,Oru njettalum illa romanjavum illa ayye,4
3169,3169,Adhu ennavo namma Tamizh rasigalukku MAMMOOTT...,1
2840,2840,Suraj ettan vere level... 1:01 Soubin ikka ...,0
1217,1217,പടം പൊളിക്കും HBD mammokka By Die hard ...,0
3507,3507,Urutaam enna dialogue parayenda tone sheriyal...,3


In [47]:

# post = pd.read_csv("positive_add1and2.csv",sep=',')
# neg = pd.read_csv("negative_add_1and2.csv",sep=',')

# data = []
# labels = []

# for index, row in post.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(1)


# for index, row in neg.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(0)
  

# len(data), len(labels)

# data, labels  = shuffle(data, labels, random_state = 40)

# data[0], labels[0]

### Tokenizer

In [48]:
tokenizer = get_tokenizer('basic_english')


def tokenized_tensor(data):

  output_tokenized = []

  for sentence in data:
    output = []
    tokenized = tokenizer(sentence)
    
    for word in tokenized:
      if word in word2idx:
        id = word2idx[word]
        output.append(id)
      else:
        word2idx[word] = len(word2idx)
        id = word2idx[word]
        output.append(id)

    output = torch.tensor(output)


    output_tokenized.append(output)

  return output_tokenized

In [49]:
# tokenized_sequences = tokenized_tensor(data)

train_tokenized_sequences = tokenized_tensor(train_data)

test_tokenized_seuqences = tokenized_tensor(test_data)

val_tokenized_seuquences = tokenized_tensor(val_data)


In [50]:
print(train_tokenized_sequences[1])

tensor([1112, 1113, 1114, 1115, 1116,  349,  114,  654, 1117, 1118])


In [51]:
word2idx['<PAD>'] = len(word2idx)
word2idx['<PAD>']

13510

In [52]:
len(word2idx)

13511

In [53]:
## Create embedding matrix

random_init = torch.nn.Parameter(torch.Tensor( (len(word2idx) - vocab_size), vocab_dim))
torch.nn.init.kaiming_uniform_(random_init, a=math.sqrt(5))


new_matrix = np.zeros( (len(word2idx), vocab_dim) )

new_matrix[:vocab_size, :] = embedding_matrix

embedding_matrix = new_matrix

embedding_matrix[vocab_size:, :] = random_init.detach().numpy()

In [54]:
embedding_matrix.shape

(13511, 100)

In [55]:
# padded_sequences = pad_sequence(tokenized_sequences, batch_first= True, padding_value=107512)

train_padded_sequences = pad_sequence(train_tokenized_sequences, batch_first= True, padding_value=word2idx['<PAD>'])

val_padded_sequences = pad_sequence(val_tokenized_seuquences, batch_first= True, padding_value=word2idx['<PAD>'])

test_padded_sequences = pad_sequence(test_tokenized_seuqences, batch_first= True, padding_value=word2idx['<PAD>'])

In [56]:
(train_padded_sequences[1])

tensor([ 1112,  1113,  1114,  1115,  1116,   349,   114,   654,  1117,  1118,
        13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
        13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
        13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
        13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
        13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
        13510])

### Dataset and Data loader 

In [57]:
class Dataset(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load the text and their corresponding labels into Pytorch tensors
    """
    def __init__(self, sequences, labels):
        self.labels = labels
        self.sequences = sequences

    def __getitem__(self, idx):
        sample = {}
        sequence = self.sequences[idx]
        label = torch.tensor(self.labels[idx])

        try:
            sample["label"] = label
            sample["token"] = sequence
        except Exception as e:
            print(e)
        
        return sample
    
    def __len__(self):
        return len(self.labels)
        

In [58]:
train_dataset = Dataset(train_padded_sequences, train_labels)

val_dataset = Dataset(val_padded_sequences, val_labels)

test_dataset = Dataset(test_padded_sequences, test_labels)

### Hyper parameters

In [59]:
## Hyper parameter

vocab_size = len(word2idx)
embed_dim = vocab_dim
seq_len = 50
hidden_size = 512
num_layer = 3
num_class = 7
batch_size = 32

LEARNING_RATE = 1e-3
EPOCHS = 30
CLIP = 0.3

In [60]:
# # Create datasets
# dataset = Dataset(padded_sequences, labels)

# split = 0.85
# train_size = int(split*len(dataset))
# val_size = len(dataset) - train_size

# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [61]:

## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

## For testing
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )


dataloaders = {'Train': train_loader, 'Val': val_loader}



In [62]:
len(train_loader), len(val_loader), len(test_loader)

(125, 12, 14)

In [63]:
max_len = -1
for comment in val_dataset:
  sentence = comment['token']
  max_len = max(max_len,sentence.shape[0])

print(max_len)

44


In [64]:
train_dataset[0]

{'label': tensor(4),
 'token': tensor([    1,  1109,    32,  1110,    32,  1111, 13510, 13510, 13510, 13510,
         13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
         13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
         13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
         13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
         13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510, 13510,
         13510])}

### Model

In [65]:
class SelfMatchingLayer(nn.Module):

    def __init__(self,  seq_length, embed_dim, **kwargs):

      super(SelfMatchingLayer, self).__init__()

      self.seq_length = seq_length
      self.embed_dim  = embed_dim

      self.P = torch.nn.Parameter(torch.Tensor(self.embed_dim, self.embed_dim))

      self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.P, a=math.sqrt(5))

      
    def forward(self, x):  
      
      # input shape: [batch, seq_len, embed_dim]


      #---------------------------------------------#
      # calculate weight vector a = {e_i . P.Q . e_j}
      #---------------------------------------------#

      out = torch.matmul(x,  self.P)   #out shape: [batch, seq_len, embed_dim]

      out = torch.matmul(out, torch.transpose(x, 1, 2))   #out shape: [batch, seq_len, seq_len]

      out = F.gelu(out)         # apply non linear activation

      #------------------------------------#
      # take row wise mean and apply softmax
      #------------------------------------#
      out = torch.mean(out, 2)  #out shape: [batch, seq_len, seq_len]

      out = torch.softmax(out, 0)     #out shape: [batch, seq_len, seq_len]

      out = out.unsqueeze(1)          #out shape: [batch, 1, seq_len]

      #-------------------------------------------#
      # calculate weighted embedding of every word
      #-------------------------------------------#
      out = torch.matmul(out, x)

      out = out.squeeze(1)

      return out      #out shape: [batch, seq_len]


In [66]:
word2idx['<PAD>']

13510

In [67]:
class SelfNet(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class):
        super(SelfNet, self).__init__()


        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = word2idx['<PAD>'])
        self.embedding.load_state_dict({'weight': torch.from_numpy(embedding_matrix)})
        self.embedding.weight.requires_grad = True

        self.selfnet_layer = SelfMatchingLayer(seq_len, embed_dim)

        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = num_layer, dropout = 0.3, bidirectional = True, batch_first = True )


        self.fc1 = nn.Linear(2* hidden_size + embed_dim , hidden_size//4)
        self.fc2 = nn.Linear(hidden_size//4, num_class)

        self.dropout = nn.Dropout(0.3)



    def forward(self, input):

        embedded = self.embedding(input)  #out shape = [batch, seq_len, embed_dim] 

        selfmatch_output = self.selfnet_layer(embedded)  #out shape = [batch, seq_len] 

        lstm_out, _ = self.lstm(embedded)     

        lstm_out = lstm_out[:, -1, :]      #out shape = [batch, 2 * hidden_size]      

        concat = torch.cat( (selfmatch_output, lstm_out), 1)     #out shape = [batch, 2 * hidden_size + seq_len ]      

        linear_out = self.dropout(F.relu(self.fc1(concat)))     #out shape = [batch, hidden_size]      

        final_out = self.fc2(linear_out)     #out shape = [batch, 2]      

        return final_out

In [68]:

### Test
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)
model = model.to(device)

# for batch in train_loader:
#   x = batch['token'].to(device)
#   out = model(x)
#   print(out.shape)
#   break



In [69]:
model

SelfNet(
  (embedding): Embedding(13511, 100, padding_idx=13510)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

### Optimizer and loss

In [70]:
#optimizer
optimizer = Adam(model.parameters(), lr = LEARNING_RATE, eps=1e-8)
#Loss function
criterion = nn.CrossEntropyLoss()

In [71]:
#to calculate accuracy

def get_accuracy(preds, labels):
  total_acc = 0.0
  
  for i in range(len(labels)):
    if labels[i] == preds[i]:
      total_acc+=1.0
  
  return total_acc / len(labels)

### Training

In [72]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.2, patience=5, threshold=0.0008, verbose = True)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5, verbose = True)


Adjusting learning rate of group 0 to 1.0000e-03.


In [73]:
PATH = '/home/ckm/ck_project/codemix/code/selfnet_mihir/models/selfnet_mean+gelu_2_mihir_yt_1.pt'

In [74]:
best_valid_f1 = 0.0000

for epoch in range(0, EPOCHS):
  

    print('-'*50)
    print('Epoch {}/{}'.format(epoch+1, EPOCHS))

    for phase in ['Train', 'Val']:

        loss = 0.0   #epoch loss
        accuracy = 0.0   #epoch accuracy

        y_true = []
        y_pred = []

        if phase == 'Train':
            model.train()
        else:
            model.eval()
        
        with tqdm(dataloaders[phase], unit="batch") as tepoch:

          for batch in tepoch:
            labels = batch["label"].to(device)
            text = batch["token"].to(device)
            

            output = model(text)

            loss = criterion(output, labels)

            if phase == 'Train':

                #zero gradients
                optimizer.zero_grad() 

                # Backward pass  (calculates the gradients)
                loss.backward()   

                # gradient clipping
                nn.utils.clip_grad_norm_(model.parameters(), CLIP)    

                optimizer.step()             # Updates the weights    

            sleep(0.1)
            _, preds = output.data.max(1)
            y_pred.extend(preds.tolist())
            y_true.extend(labels.tolist())
            
            batch_acc = get_accuracy(preds.tolist(), labels.tolist())
            
            
            loss += loss.item()
            accuracy+= batch_acc

              
          epoch_loss = loss / (len(dataloaders[phase]))
          epoch_acc = accuracy / (len(dataloaders[phase]))

          print(phase + ":")
          
          
          #print(confusion_matrix(y_true, y_pred))
          pre = precision_score(y_true, y_pred, average='weighted')
          recall = recall_score(y_true, y_pred, average='weighted')
          f1 = f1_score(y_true, y_pred, average='weighted')
          

          print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))
          # save best model
          print()
          
            
          if phase == 'Val':
                
                if f1 > best_valid_f1:
                    best_valid_f1 = f1
                    
                    torch.save(model.state_dict(), PATH)
                    print('Model Saved!')
                    
                scheduler.step()
                    


--------------------------------------------------
Epoch 1/30


100%|██████████| 125/125 [01:07<00:00,  1.85batch/s]
/home/ckm/python3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train:
F1: 0.2948, Precision: 0.2837, Recall : 0.3985, Accuracy: 0.3985, Loss: 0.0233.



100%|██████████| 12/12 [00:17<00:00,  1.42s/batch]
/home/ckm/python3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Val:
F1: 0.2583, Precision: 0.1846, Recall : 0.4297, Accuracy: 0.4297, Loss: 0.2176.

Model Saved!
Adjusting learning rate of group 0 to 5.0000e-04.
--------------------------------------------------
Epoch 2/30


...

100%|██████████| 125/125 [00:59<00:00,  2.09batch/s]
/home/ckm/python3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train:
F1: 0.4770, Precision: 0.4201, Recall : 0.5680, Accuracy: 0.5680, Loss: 0.0157.



100%|██████████| 12/12 [00:17<00:00,  1.44s/batch]

Val:
F1: 0.3323, Precision: 0.2950, Recall : 0.4349, Accuracy: 0.4349, Loss: 0.2340.

Adjusting learning rate of group 0 to 9.3132e-13.



/home/ckm/python3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Testing

In [75]:
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)

In [76]:
model.load_state_dict(torch.load(PATH))

model.to(device)


SelfNet(
  (embedding): Embedding(13511, 100, padding_idx=13510)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [77]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 14/14 [00:17<00:00,  1.27s/batch]


Inference:

[[145   0  58   0   0]
 [ 13   0  50   0   0]
 [ 31   0  86   0   0]
 [ 15   0   9   0   0]
 [ 15   0  26   0   0]]

F1: 0.5156, Precision: 0.5156, Recall : 0.5156, Accuracy: 0.6016, Loss: 0.2322.


In [78]:
model.load_state_dict(torch.load(PATH))

model.to(device)


SelfNet(
  (embedding): Embedding(13511, 100, padding_idx=13510)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [79]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 14/14 [00:19<00:00,  1.37s/batch]


Inference:

[[143   0  59   0   0]
 [ 15   0  48   0   0]
 [ 23   0  94   0   0]
 [ 10   0  15   0   0]
 [ 17   0  24   0   0]]

F1: 0.4521, Precision: 0.4123, Recall : 0.5290, Accuracy: 0.6172, Loss: 0.2004.



/home/ckm/python3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 14/14 [00:17<00:00,  1.27s/batch]


Inference:

[[141   0  61   0   0]
 [ 12   0  51   0   0]
 [ 29   0  88   0   0]
 [  9   0  16   0   0]
 [ 14   0  27   0   0]]

F1: 0.5112, Precision: 0.5112, Recall : 0.5112, Accuracy: 0.5964, Loss: 0.2325.
